In [1]:
from datetime import datetime
from urllib.error import HTTPError

from langchain_core.messages.tool import tool_call

print("Hello, world!")

Hello, world!


In [2]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 195.4 kB/s eta 0:00:0000:0100:03
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.169.0
    Uninstalling google-api-python-client-2.169.0:
      Successfully uninstalled google-api-python-client-2.169.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [google-api-python-client]api-python-client]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [3]:
!pip install google-ai-generativelanguage==0.6.15

  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 262.1 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.67.1
    Uninstalling grpcio-1.67.1:
      Successfully uninstalled grpcio-1.67.1
  Attempting uninstall: google-ai-generativelanguage 0/2 [grpcio]
    Found existing installation: google-ai-generativelanguage 0.6.18o]
    Uninstalling google-ai-generativelanguage-0.6.18:m0/2 [grpcio]
      Successfully uninstalled google-ai-generativelanguage-0.6.18cio]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [google-ai-generativelanguage]enerativelanguage]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymilvus 2.5.11 requires grpcio<=1.67.1,>=1.49.1, 

In [3]:
# # Install all required packages with specific versions in Colab
# !pip install google-api-python-client==2.171.0 \
# google-auth-httplib2==0.2.0 \
#     google-auth-oauthlib==1.2.2 \
#     google-ai-generativelanguage==0.6.15 \
#     google-generativeai==0.8.5

In [68]:
import os
import datetime
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.errors import HttpError
from langchain_core.tools import tool

SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]

@tool
def read_calendar(date: str, days_before: int = 2, days_after: int = 2) -> list:
    """Read the calendar using the Calendar API and return events within a date range.
    Args:
        date: Central date to check (YYYY-MM-DD format)
        days_before: Number of days before the central date to include (default 2)
        days_after: Number of days after the central date to include (default 2)
    Returns:
        list: A list of events from the calendar with start times and summaries.
    """
    # Authentication
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
            creds = flow.run_local_server(port=8088)
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("calendar", "v3", credentials=creds)
        all_events = []

        # Set timezone to IST (UTC+5:30)
        tz = datetime.timezone(datetime.timedelta(hours=5, minutes=30))
        try:
            central_date = datetime.datetime.strptime(date, "%Y-%m-%d").replace(tzinfo=tz)
        except ValueError:
            return [{"error": "Invalid date format. Please use YYYY-MM-DD"}]

        # Calculate date range
        start_date = central_date - datetime.timedelta(days=days_before)
        end_date = central_date + datetime.timedelta(days=days_after + 1)  # +1 to include the full last day

        start = start_date.replace(hour=0, minute=0, second=0).isoformat()
        end = end_date.replace(hour=23, minute=59, second=59).isoformat()

        print(f"Searching between {start_date.date()} and {end_date.date()} (India Time)")

        # Get list of all calendars
        try:
            calendar_list = service.calendarList().list().execute()
            calendars = calendar_list.get('items', [])
        except HttpError as e:
            print(f"Error getting calendars: {e}")
            calendars = []

        # Always check primary calendar first
        calendars.insert(0, {'id': 'primary', 'summary': 'Primary Calendar'})

        for calendar in calendars:
            calendar_id = calendar['id']
            print(f"\nChecking calendar: {calendar.get('summary', 'Unknown')}")

            try:
                events_result = service.events().list(
                    calendarId=calendar_id,
                    timeMin=start,
                    timeMax=end,
                    singleEvents=True,
                    orderBy="startTime"
                ).execute()
                events = events_result.get('items', [])
                print(f"Found {len(events)} events")

                # Format events before adding
                for event in events:
                    formatted_event = {
                        'summary': event.get('summary', 'No title'),
                        'start': event.get('start', {}),
                        'end': event.get('end', {}),
                        'calendar': calendar.get('summary', 'Unknown'),
                        'date': event['start'].get('dateTime', event['start'].get('date'))[:10]  # Extract YYYY-MM-DD
                    }
                    all_events.append(formatted_event)

            except HttpError as e:
                print(f"Error accessing calendar {calendar_id}: {e}")
                continue

        if not all_events:
            no_event_message = {
                'summary': f'No events found between {start_date.date()} and {end_date.date()}',
                'date_range': {
                    'start': start_date.date().isoformat(),
                    'end': end_date.date().isoformat()
                },
                'note': 'Checked all available calendars'
            }
            return [no_event_message]

        # Sort events by date
        all_events.sort(key=lambda x: x['date'])

        return all_events

    except HttpError as e:
        return [{"error": f"Calendar API error: {str(e)}"}]
    except Exception as e:
        return [{"error": f"Unexpected error: {str(e)}"}]

In [69]:
# Example usage
if __name__ == "__main__":
    date_to_check = "2025-06-13"
    days=10
    events = read_calendar(date_to_check,days,days)

    print("\nResults:")
    for event in events:
        if 'error' in event:
            print(f"Error: {event['error']}")
        elif 'start' in event:
            start = event["start"].get("dateTime", event["start"].get("date", "No start time"))
            print(f"{start}: {event['summary']} (Calendar: {event.get('calendar', 'Unknown')})")
        else:
            print(f"No events found in the specified date range")

TypeError: BaseTool.__call__() takes from 2 to 3 positional arguments but 4 were given